In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv(r'../data/sales.csv', index_col=0)

In [3]:
import sklearn
import hts

In [4]:
data.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,month_year,year
0,Product_0993,Whse_J,Category_028,2012-07-27,100.0,2012-07,2012
1,Product_0979,Whse_J,Category_028,2012-01-19,500.0,2012-01,2012
2,Product_0979,Whse_J,Category_028,2012-02-03,500.0,2012-02,2012
3,Product_0979,Whse_J,Category_028,2012-02-09,500.0,2012-02,2012
4,Product_0979,Whse_J,Category_028,2012-03-02,500.0,2012-03,2012


In [5]:
data["Warehouse"].unique()

array(['Whse_J', 'Whse_S', 'Whse_C', 'Whse_A'], dtype=object)

In [6]:
# abbreviating warehouse and product category
mapping_warehouse ={
    "Whse_J" : "WJ",
    "Whse_S" : "WS",
    "Whse_C": "WC",
    "Whse_A": "WA",
    
}
data["Warehouse"] = data["Warehouse"].map(mapping_warehouse)

In [7]:
data["Product_Category"].unique()

array(['Category_028', 'Category_006', 'Category_001', 'Category_019',
       'Category_005', 'Category_030', 'Category_032', 'Category_011',
       'Category_033', 'Category_007', 'Category_015', 'Category_021',
       'Category_024', 'Category_026', 'Category_023', 'Category_022',
       'Category_009', 'Category_003', 'Category_004', 'Category_018',
       'Category_020', 'Category_013', 'Category_008', 'Category_017',
       'Category_031', 'Category_025', 'Category_010', 'Category_029',
       'Category_012', 'Category_002', 'Category_014', 'Category_027',
       'Category_016'], dtype=object)

In [8]:
# mapping_category={
#     'Category_028': 'Cat28',
#     'Category_006': 'Cat06',
#     'Category_001': 'Cat01', 
#     'Category_019':'Cat19',
#        'Category_005', 
#        'Category_030', 
#        'Category_032', 
#        'Category_011',
#        'Category_033', 
#        'Category_007', 
#        'Category_015', 
#        'Category_021',
#        'Category_024', 
#        'Category_026', 
#        'Category_023', 
#        'Category_022',
#        'Category_009', 
#        'Category_003', 
#        'Category_004', 
#        'Category_018',
#        'Category_020', 'Category_013', 'Category_008', 'Category_017',
#        'Category_031', 'Category_025', 'Category_010', 'Category_029',
#        'Category_012', 'Category_002', 'Category_014', 'Category_027',
#        'Category_016
# }

In [13]:
mapping_cat = {
    
}
categories = data["Product_Category"].unique()
for i in categories:
    abbre = i[:3] + i.split("_", 1)[1]
    mapping_cat[i] = abbre

In [14]:
mapping_cat

{'Category_028': 'Cat028',
 'Category_006': 'Cat006',
 'Category_001': 'Cat001',
 'Category_019': 'Cat019',
 'Category_005': 'Cat005',
 'Category_030': 'Cat030',
 'Category_032': 'Cat032',
 'Category_011': 'Cat011',
 'Category_033': 'Cat033',
 'Category_007': 'Cat007',
 'Category_015': 'Cat015',
 'Category_021': 'Cat021',
 'Category_024': 'Cat024',
 'Category_026': 'Cat026',
 'Category_023': 'Cat023',
 'Category_022': 'Cat022',
 'Category_009': 'Cat009',
 'Category_003': 'Cat003',
 'Category_004': 'Cat004',
 'Category_018': 'Cat018',
 'Category_020': 'Cat020',
 'Category_013': 'Cat013',
 'Category_008': 'Cat008',
 'Category_017': 'Cat017',
 'Category_031': 'Cat031',
 'Category_025': 'Cat025',
 'Category_010': 'Cat010',
 'Category_029': 'Cat029',
 'Category_012': 'Cat012',
 'Category_002': 'Cat002',
 'Category_014': 'Cat014',
 'Category_027': 'Cat027',
 'Category_016': 'Cat016'}

In [ ]:
# build hierarchy
total = {'month_year': list(categories)}
category = {i: [j for j in products if j.startswith(i)] for i in categories}
#product = {i: [j for j in products]}
hierarchy = {**total, **category}